# Import

In [62]:
# import yfinance as yf
import pandas as pd
import os
from os.path import join
from datetime import datetime, timezone, timedelta
import pandas as pd
from yforseer.scraping import update_raw_tables, load_ticker_list, RawDF_Schema
import matplotlib.pyplot as plt
import numpy as np
import logging
import torch
from torch.utils.data import DataLoader, Dataset


# Dataset and loader

In [12]:
load_array_pth = 'data/yahoo/artifacts/data_array.npz'

data = np.load(load_array_pth)

In [91]:
class StockDataset(Dataset):
    def __init__(self, data, memory, lookahead):
        self.data = data  # (N_tickers, N_days), torch.tensor, float32
        self.memory = memory
        self.lookahead = lookahead
        self.window = memory + lookahead
        self.n_tickers = data.shape[0]
        self.n_days = data.shape[1]
        self.final_start_ind = self.n_days - self.window
        self.n_samples = self.final_start_ind + 1
        
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        
        start, mid, end = idx, idx+self.memory, idx+self.window
        if end > self.n_days:
            raise IndexError('Index out of bounds')
        X = self.data[:, start:mid]
        y_interval = self.data[:, mid:end]
        ylast = y_interval[:, [-1]]
        ymin = y_interval.min(dim=1, keepdim=True)[0]   
        ymax = y_interval.max(dim=1, keepdim=True)[0]   
        ymean = y_interval.mean(dim=1, keepdim=True)
        y = torch.cat([ylast, ymin, ymax, ymean], dim=1)
        return X, y

In [92]:

price_array = torch.from_numpy(data['data'])



In [100]:
dataset = StockDataset(price_array, 60, 30)
for Xbatch, ybatch in DataLoader(dataset, batch_size=32):
    print(Xbatch.shape, ybatch.shape)

torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 60]) torch.Size([32, 37, 4])
torch.Size([32, 37, 

In [84]:
foo = torch.rand(10)

foo[6:12]

tensor([0.2196, 0.9866, 0.6898, 0.0496])